In [10]:
import numpy as np
import pandas as pd

In [9]:
import subprocess

In [64]:
rnarobo_=open("rnarobo_result_dereplicated.txt")
rnarobo=rnarobo_.readlines()
rnarobo_.close()

In [65]:
trna_=open("combined_trna_scan_v2.txt")
trna=trna_.readlines()
trna_.close()

In [66]:
#rnarobo[11:-6] gives us the part we need

In [74]:
#trna[3:] gives us the part we need

In [66]:
dicti={}
for line in trna[3:]:
    #create a dictionary with seq name as key and the starting position as value
        line=line.strip().split()
        if line[0] in dicti:
            dicti[line[0]].append(int(line[3]))
        else:
            dicti[line[0]]=[int(line[3])]
            
        
        

In [67]:
dicti_r={}
for line in rnarobo[11:-6]:
    if line[0]!="|":
        line=line.replace(".","_")
        line=line.strip().split()
        if line[2] in dicti_r:
            dicti_r[line[2]].append(int(line[0]))
        else:
            dicti_r[line[2]]=[int(line[0])]
        

In [68]:
matching=[]
for key in dicti_r:
    if key in dicti:
        for value in dicti_r[key]:
            for value2 in dicti[key]:
                if value-value2 in range(-5,6):
                    matching.append([key,value,value2])

In [69]:
#make sure everything is according to plan and we have no duplicates etc.
newm=[]
for mat in matching:
    if mat not in newm:
        newm.append(mat)

In [70]:
lets_do_this=pd.DataFrame(columns=["ID","trna_nr","trna_start","trna_end","trna_type","codon","ins_start","ins_end","trna_confidence","psuedo","rnarobo_start","rnarobo_end","rnarobo_seq"])
check=0
duplicates=[]
for nr in range(len(newm)):
    match=newm[nr]
    lin_df=""
    for line in trna[3:]:
    #create a dictionary with seq name as key and the starting position as value
        line=line.strip().split()
        
        if match[0]==line[0]:
            if len(lin_df.split())==0:
                if str(match[2])==line[3]:
                    #print(line)
                    lin_df+=" ".join(line)+" "
                    if lin_df[-2]!="o":
                        lin_df+="NA "
            else: 
                duplicates.append(line)
    for line in rnarobo[11:-6]:
        line=line.replace(".","_")
        line=line.strip().split()
        if len(lin_df.split())<13:

            if check==1:
                #if len(lin_df.split())<13:
                    lin_df+=str(line[0])
                    check=0
                #else:
                        #duplicates.append(line)

                #check=0
             #####FIX 2: Important here: we did this to make sure that we are actually matching the right ribozyme here...
            if len(line)>1:
                #here we made the mistake: not checking the actual ID but only starting position
                if match[0]==line[2]:
                    if str(match[1])==line[0]:
                    #if len(lin_df.split())<13:
                        lin_df+=" ".join(line[:2])
                        lin_df+=" "
                        check=1
            #else:
                #duplicates.append(line)
                #check=0
    lin_f=lin_df.split()
    lets_do_this.loc[nr]=np.array(lin_f)
    #lets_do_this.iloc[nr,:8]=lin_df[0]
    #lets_do_this.iloc[nr,8:-1]=lin_df[1]
    #lets_do_this.iloc[nr,-1]=lin_df[2]

In [80]:
#save intermediate results and reload: automatically fixes correct datatype (int, etc)
df1str=lets_do_this.to_csv(sep="\t")
f= open("comb_trna_v2_plus_ribo.csv","w+")
f.write(df1str)
f.close()

fff=pd.read_csv("comb_trna_v2_plus_ribo.csv",sep="\t",index_col=0)

df1str=lets_do_this.to_csv(sep="\t")
f= open("comb_trna_v2_plus_ribo.csv","w+")
f.write(df1str)
f.close()

In [22]:
#fff=pd.read_csv("comb_trna_v2_plus_ribo.csv",sep="\t",index_col=0)

In [81]:
fff["trnaseq"]= " "

fff["ribozyme_control"]= " "

In [82]:
#lets try to save and cut out the respective trnas (and in between part)
for ID in range(len(fff)):
        #select the correct contig/genome from the rnarobo hit
        inp="grep -A1 -m1 '"+fff.iloc[ID,0]+"' combined_all_viral_sequences_v2.fasta" 
        res=subprocess.run(inp,shell=True, capture_output=True, text=True)
        sequ=res.stdout.split()[-1]
        start=fff.iloc[ID,2]-1
        end=fff.iloc[ID,3]
        start2=fff.iloc[ID,10]-1
        end2=fff.iloc[ID,11]
        if start < end:
            trna=sequ[start:end]
            rnarobo=sequ[start2:end2]
            fff.iloc[ID,-2]=trna
            fff.iloc[ID,-1]=rnarobo
        if end < start:
            #reverse sequence: add +1 to reverse previous indexing correction 
            end-=1
            start+=1
            end2-=1
            start2+=1
            trna_rev=sequ[end:start]
            ribo_rev=sequ[end2:start2]
            trna=[]
            rnarobo=[]
            #reverse parts: transcribe back into the correct format!
            for nuc in range(len(trna_rev)-1,-1,-1):
                if trna_rev[nuc]=="A":
                    trna.append("T")
                if trna_rev[nuc]=="T":
                    trna.append("A")
                if trna_rev[nuc]=="G":
                    trna.append("C")
                if trna_rev[nuc]=="C":
                    trna.append("G")
                if trna_rev[nuc]=="N":
                    trna.append("N")
            for nuc in range(len(ribo_rev)-1,-1,-1):
                if ribo_rev[nuc]=="A":
                    rnarobo.append("T")
                if ribo_rev[nuc]=="T":
                    rnarobo.append("A")
                if ribo_rev[nuc]=="G":
                    rnarobo.append("C")
                if ribo_rev[nuc]=="C":
                    rnarobo.append("G")
                if ribo_rev[nuc]=="N":
                    rnarobo.append("N")
            trna="".join(trna)
            rnarobo="".join(rnarobo)
            fff.iloc[ID,-2]=trna
            fff.iloc[ID,-1]=rnarobo

In [84]:
#check if the control and actual seq are the same
for x in range(len(fff)):
    seq1=fff.loc[x,"rnarobo_seq"]
    seq3="".join(seq1.split("|"))
    seq2=fff.loc[x,"ribozyme_control"]
    if seq2!=seq3:
        print("something wrong",x)
        print(seq2)
        print(seq3)

In [86]:
#still need to fix the introns!! :)
#if interested in this: substract the ins_start from trna_start and cut it out
fff[fff["ins_start"]!=0]

,ID,trna_nr,trna_start,trna_end,trna_type,codon,ins_start,ins_end,trna_confidence,psuedo,rnarobo_start,rnarobo_end,rnarobo_seq,trnaseq,ribozyme_control
9,SAMEA2737752_a1_ct159360_vs1@Unclassified_viru...,1,6451,6745,Lys,CTT,6488,6710,36.6,pseudo,6746,6801,|GAATAGC|GTA|AGTAGAA|ACC|T|CC|ACGT|GGT|GCTATTC...,TCTCCTATGGTGTAATGGTTAGCACAGAAGACTCTTAAAATAAGGA...,GAATAGCGTAAGTAGAAACCTCCACGTGGTGCTATTCGGATAACGC...
44,SAMN10290229_a1_ct27083_vs1@linear@Unclassifie...,1,33863,33569,Lys,CTT,33826,33604,35.7,pseudo,33568,33513,|GAATAGC|GTA|AGTAGAA|ACC|T|CC|ACGT|GGT|GCTATTC...,TCTCCTATGGTGTAATGGTTAGCACAGAAGACTCTTAAAATAAGGA...,GAATAGCGTAAGTAGAAACCTCCACGTGGTGCTATTCGGATAACGC...
142,uvig_93816,1,5029,5166,Ser,GCT,5077,5130,21.7,pseudo,5167,5223,|GGAGCAC|ATA|TTGAGAT|ACT|T|CC|ACGT|AGT|GTGCTCT...,GGAGAACCCCTAGTTTTGTTAGTTTCTATCAAAAACTGGCTATTGC...,GGAGCACATATTGAGATACTTCCACGTAGTGTGCTCTGGGTAATGC...
146,uvig_98904,2,134967,134830,Ser,GCT,134919,134866,21.7,pseudo,134829,134773,|GGAGCAC|ATA|TTGAGAT|ACT|T|CC|ACGT|AGT|GTGCTCT...,GGAGAACCCCTAGTTTTGTTAGTTTCTATCAAAAACTGGCTATTGC...,GGAGCACATATTGAGATACTTCCACGTAGTGTGCTCTGGGTAATGC...
196,uvig_208533,1,7281,7575,Lys,CTT,7318,7540,36.6,pseudo,7576,7631,|GAATAGC|GTA|AGTAGAA|ACC|T|CC|ACGT|GGT|GCTATTC...,TCTCCTATGGTGTAATGGTTAGCACAGAAGACTCTTAAAATAAGGA...,GAATAGCGTAAGTAGAAACCTCCACGTGGTGCTATTCGGATAACGC...
250,uvig_363524,1,71609,71315,Lys,CTT,71572,71350,38.6,NaN,71314,71259,|GAATAGC|GTA|AGTAGAA|ACC|T|CC|ACGT|GGT|GCTATTC...,TCTCCTATGGTGTAATGGTTAGCACAGAAGACTCTTAATATAAGGA...,GAATAGCGTAAGTAGAAACCTCCACGTGGTGCTATTCGGATAACGC...
271,uvig_502434,1,145767,146061,Lys,CTT,145804,146026,37.6,NaN,146062,146117,|GAATAGC|GTA|AGTAGAA|ACC|T|CC|ACGT|GGT|GCTATTC...,TCTCCTATGGTGTAATGGTTAGCACAGAAGACTCTTAAAATAAGGA...,GAATAGCGTAAGTAGAAACCTCCACGTGGTGCTATTCGGATAACGC...
437,MGV-GENOME-0378080,8,5616,5322,Lys,CTT,5579,5357,36.1,pseudo,5321,5266,|GAATAGC|GTA|AGTAGAA|ACC|T|CC|ACGT|GGT|GCTATTC...,TCTCCTATGGTGTAATGGTTAGCACAGAAGACTCTTAATATAAGGA...,GAATAGCGTAAGTAGAAACCTCCACGTGGTGCTATTCGGATAACGC...
444,MGV-GENOME-0378146,13,1228,934,Lys,CTT,1191,969,36.6,pseudo,933,878,|GAATAGC|GTA|AGTAGAA|ACC|T|CC|ACGT|GGT|GCTATTC...,TCTCCTATGGTGTAATGGTTAGCACAGAAGACTCTTAAAATAAGGA...,GAATAGCGTAAGTAGAAACCTCCACGTGGTGCTATTCGGATAACGC...
447,MGV-GENOME-0378207,1,172435,172729,Lys,CTT,172472,172694,36.6,pseudo,172730,172785,|GAATAGC|GTA|AGTAGAA|ACC|T|CC|ACGT|GGT|GCTATTC...,TCTCCTATGGTGTAATGGTTAGCACAGAAGACTCTTAAAATAAGGA...,GAATAGCGTAAGTAGAAACCTCCACGTGGTGCTATTCGGATAACGC...


In [168]:
f=fff.copy(deep=True)

In [ ]:
#lets check from where each of the hits are coming

In [ ]:
# >SRS101376_a1_ct26874_vs01@Myoviridae__sp._ctSu1169@linear : 1, CHVD_clustered_mash99_v1.fasta
#downloaded from: https://zenodo.org/record/4776317
#contigs downloaded, not the viral genomes...
#A catalog of tens of thousands of viruses from human
#metagenomes reveals hidden associations with chronic diseases
# https://doi.org/10.1073/pnas.2023202118 

#potentially identifying unique: @


In [ ]:
#2:
# >ivig_7	VIRSorter_QVHT01000001_1_Streptococcus_gallolyticus_strain_AM24-4_AM24-4_Scaf1__whole_genome_shotgun_sequence_gene_736_gene_790-715237-758471-cat_5
# >uvig_598943
#GDP sequences.fa

#https://www.sciencedirect.com/science/article/pii/S0092867421000726



#vig_ as identifier?


In [ ]:
#3: https://academic.oup.com/database/article/doi/10.1093/database/baw162/2742079?login=true
#one ribozyme found... refseq|...


In [ ]:
#4: stopcodon paper
#https://www.nature.com/articles/s41564-022-01128-6 
#>aot2015-NO14_SRR1761688_USA_scaffold_60
#>WM-2_scaffold_8

#scaffold as unique?


In [ ]:
#5
#MDV 
#https://www.nature.com/articles/s41564-021-00928-6#Abs1


#>MGV-GENOME-0345437 
#>MGV-GENOME-0373928

#unique idetifier: MGV



In [ ]:
#6: >Altabtbaei_2021_02.k141_296648

#https://www.sciencedirect.com/science/article/pii/S2589004222006897


#>Zhang_2015_265.ctg_253_1 18718-38830/38830

#maybe no unique idetifiers?

In [ ]:
#7:  https://www.sciencedirect.com/science/article/pii/S0092867419303411

#GOV_all_contigs


#>GOV_bin_1784_contig-100_68


#unique feature: GOV



In [ ]:
#8: 

#https://genome.jgi.doe.gov/portal/IMG_VR/IMG_VR.home.html
#https://academic.oup.com/nar/article/49/D1/D764/5952208?login=true


#>IMGVR_UViG_2061766007_001007|2061766007|rumenHiSeq_NODE_4331347_len_22405_cov_0_883553



#unique: IMGCR




#new version available since jan 23- v4... but maybe lets stick with v3 for now!

In [169]:
#create new columns for study origin
f.loc[:,"study_origin"]=""

for row in range(len(f)):
    idz=f.loc[row,"ID"]
    if "@" in idz:
        f.loc[row,"study_origin"]="1"
    elif "vig_" in idz:
        f.loc[row,"study_origin"]="2"
    elif "refseq|" in idz:
        f.loc[row,"study_origin"]="3"
    elif "scaffold" in idz:
        f.loc[row,"study_origin"]="4"
    elif "MGV" in idz:
        f.loc[row,"study_origin"]="5"
    elif "idontknow" in idz:
        f.loc[row,"study_origin"]="6"
    elif "GOV" in idz:
        f.loc[row,"study_origin"]="7"
    elif "IMGVR" in idz:
        f.loc[row,"study_origin"]="8"

In [170]:
f.loc[0:59,"study_origin"]="1"

In [171]:
f.loc[59:299,"study_origin"]="2"

In [ ]:
#three: not found

In [172]:
f.loc[299:315,"study_origin"]="4"

In [173]:
f.loc[315:679,"study_origin"]="5"

In [174]:
f.loc[679:680,"study_origin"]="6"

In [ ]:
#7 not present

In [175]:
f.loc[681:,"study_origin"]="8"

In [176]:
f["study_origin"].value_counts()

5    364
2    240
8     61
1     59
4     16
6      2
Name: study_origin, dtype: int64

In [177]:
f

,ID,trna_nr,trna_start,trna_end,trna_type,codon,ins_start,ins_end,trna_confidence,psuedo,rnarobo_start,rnarobo_end,rnarobo_seq,trnaseq,ribozyme_control,study_origin
0,SAMEA1906455_a1_ct67271_vs1@Unclassified_virus...,10,8561,8478,Leu,TAA,0,0,53.4,pseudo,8477,8423,|GAATAGC|AA|ATGAGAA|ACC|T|CC|ACGT|GGT|GCTATTT|...,ACCCCTTTATCCGAACAGGTATAGGAGATGGTCTTAAACACCATTT...,GAATAGCAAATGAGAAACCTCCACGTGGTGCTATTTGGGTAATGCT...,1
1,SAMEA1906455_a1_ct67271_vs1@Unclassified_virus...,1,17583,17654,Sup,CTA,0,0,60.8,NaN,17655,17710,|GAATAGC|GTA|AGTAGAA|ACC|T|CC|ACGT|GGT|GCTATTC...,TCTCCTATGGTGTAATGGTTAGCACAGAAGACTCTAAATCTTTTAG...,GAATAGCGTAAGTAGAAACCTCCACGTGGTGCTATTCGGATAACGC...,1
2,SAMEA1906581_a1_ct61757_vs1@Siphoviridae__sp__...,3,113271,113199,Met,CAT,0,0,53.9,NaN,113198,113143,|GAGTTGC|AG|TTAGTCA|ACC|T|CC|ACGT|GGT|GCAACTT|...,CGCGGGGTATAGCAACAGGTAGCTTGTCAGTCTCATAATCTGAAGG...,GAGTTGCAGTTAGTCAACCTCCACGTGGTGCAACTTGGATTAAAAC...,1
3,SAMEA2580118_a1_ct141914_vs1@Podoviridae__sp__...,16,33852,33767,Leu,TAA,0,0,55.7,NaN,33766,33712,|GAAACAC|AA|GATTGAA|ACC|T|CC|ACGT|GGT|GTGTTTT|...,GCCAGAATGGTGGAATTGGTAGACACGTTAGAATTAAGATCTAATG...,GAAACACAAGATTGAAACCTCCACGTGGTGTGTTTTGGGTAAAGCT...,1
4,SAMEA2580118_a1_ct141914_vs1@Podoviridae__sp__...,9,41510,41431,Met,CAT,0,0,42.9,NaN,41430,41376,|GATTCAC|AT|AGTAAGA|ACC|T|CC|ACGC|GGT|GTGAATC|...,CGTGGAGTAGAGAAGTTGGTCATCTCGCTAGGCTCATAACCTAGAA...,GATTCACATAGTAAGAACCTCCACGCGGTGTGAATCGGGTAATGCT...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,IMGVR_UViG_3300008255_000097|3300008255|Ga0100...,2,185586,185514,His,GTG,0,0,69.7,NaN,185515,185460,|GACTGCA|GA|GAGTGCC|ACC|T|CC|CCGT|GGT|TGCAGTT|...,TTGGTCGTAGCTCAGTGGCAGAGCGCCTGACTGTGAATCAGGTGAC...,GACTGCAGAGAGTGCCACCTCCCCGTGGTTGCAGTTGGGTAACGCT...,8
738,IMGVR_UViG_3300009180_000043|3300009180|Ga0114...,1,35971,35887,Leu,TAA,0,0,80.4,NaN,35884,35828,|GGGAATT|TTT|GCAGAAC|GCC|T|CC|TCGT|GGC|GATTCCT...,GCATGGATGGCCGAGCTTGGTTAAGGCGGAAGACTTAAGATCTTCT...,GGGAATTTTTGCAGAACGCCTCCTCGTGGCGATTCCTGGGTAACTC...,8
739,Manrique_Person1_NODE_5_length_177432_cov_94_6...,20,52126,52051,Met,CAT,0,0,43.1,NaN,52050,51994,|GAGTATA|AAG|ATAGTAT|ACC|T|CC|ACGT|GGT|TATACTT...,GGCGGGATAGACTGGAGATGGTTCCCAGCTCGGTCTCATAAGCCGA...,GAGTATAAAGATAGTATACCTCCACGTGGTTATACTTGGGTAACGC...,8
740,Monaco_MBA4031_NODE_1_length_174570_cov_57_602934,14,172052,171981,Sup,CTA,0,0,56.4,NaN,171980,171925,|GAATAGC|AAA|AGTAGAA|ACC|T|CC|ACGT|GGT|GCTATTT...,TCTCTTATGGTGTAATGGTTAGCACAGAAGACTCTAAATCTTTTAG...,GAATAGCAAAAGTAGAAACCTCCACGTGGTGCTATTTGGGTAATGC...,8


In [178]:
print ("Unique ribos: "+str(len(f["ribozyme_control"].unique())))

Unique ribos: 126


In [179]:
print ("Unique trnas: "+str(len(f["trnaseq"].unique())))

Unique trnas: 152


In [100]:
#combinme ribo+trnaseq
f["comb_seq"]=f["ribozyme_control"]+f["trnaseq"]

In [101]:
print ("Unique ribo/trna_combinations: "+str(len(f["comb_seq"].unique())))

Unique ribo/trna_combinations: 185


In [102]:
print ("Unique ribos: "+str(len(f["ribozyme_control"].unique())))

Unique ribos: 126


In [159]:
df1str=f.to_csv(sep="\t",index=False)
f= open("comb_trna_seq_plus_ribo_fix_march.csv","w+")
f.write(df1str)
f.close()

In [ ]:
#okay, now lets open the ribozyme controls and check which of them are unique for 8